In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

print('pandas version is {}.'.format(pd.__version__))
print('numpy version is {}.'.format(np.__version__))
print('scikit-learn version is {}.'.format(sklearn.__version__))
print('seaborn version is {}.'.format(sns.__version__))
print('matplotlib version is {}.'.format(matplotlib.__version__))

pandas version is 0.18.0.
numpy version is 1.10.4.
scikit-learn version is 0.17.1.
seaborn version is 0.7.1.
matplotlib version is 1.5.1.


In [2]:
training_data = pd.read_csv("EH-training-data.csv", names = range(0, 59))
training_labels = pd.read_csv("EH-training-labels.csv", names = range(0, 3))
testing_data = pd.read_csv("EH-testing-data.csv", names = range(0, 59))
testing_labels = pd.read_csv("EH-testing-labels.csv", names = range(0, 3))
print "This set has {} training_data points and {} features.".format(*training_data.shape)
print "This set has {} training_label points and {} features.".format(*training_labels.shape)
print "This set has {} testing_data points and {} features.".format(*testing_data.shape)
print "This set has {} testing_label points and {} features.".format(*testing_labels.shape)
#display(training_data)
#display(training_labels)
#display(testing_data)
#display(testing_labels)

This set has 274654 training_data points and 59 features.
This set has 506688 training_label points and 3 features.
This set has 211810 testing_data points and 59 features.
This set has 439161 testing_label points and 3 features.


In [3]:
training_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
count,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.00000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.00000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000,274654.000000
mean,137327.500000,1.483204,2.336685,350.393521,1721.975285,37.532925,1.111417,1.696425,1.456407,0.337213,3.403613,4.227577,5.937125,0.311159,0.002057,770.473224,43.103279,30.217517,12.825173,3.376328,4.760448,4.176895,3.783939,0.898305,3.478023,36.30139,0.320352,0.040141,0.068490,0.011819,0.039876,0.037618,0.01913,0.027220,0.071038,0.020965,0.201373,0.237117,0.400569,0.336729,0.449886,0.235008,0.177074,0.117431,0.059981,0.110455,0.084015,0.188652,0.036581,0.021522,0.045097,0.143479,3.060094,5.708379,6.360934,5.176888,5.072466,-91.037611,38.297782
std,79285.924754,0.499719,1.722036,842.266552,99.570376,10.626110,1.320764,0.791909,0.662386,0.887937,1.342585,1.168355,17.199464,0.462968,0.045309,975.292766,12.015969,9.295457,18.015589,1.529659,1.596023,1.722220,1.796954,0.302248,2.867129,6.32139,0.466613,0.196291,0.252585,0.108069,0.195667,0.190271,0.13698,0.162723,0.256890,0.143266,0.401027,0.425315,0.490015,0.472592,0.497483,0.424005,0.381732,0.321934,0.237452,0.313457,0.277411,0.391233,0.187730,0.145116,0.207517,0.350561,1.834048,1.136248,0.908374,1.302364,1.165751,28.034269,8.739515
min,1.000000,1.000000,-3.000000,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,7.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-166.967500,-45.880000
25%,68664.250000,1.000000,3.000000,30.000000,1650.000000,29.000000,0.000000,1.000000,1.000000,0.000000,3.000000,3.000000,1.000000,0.000000,0.000000,200.000000,34.000000,23.000000,0.000000,2.000000,4.000000,3.000000,2.000000,1.000000,1.000000,32.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,6.000000,4.000000,4.000000,-111.922300,34.079900
50%,137327.500000,1.000000,3.000000,60.000000,1720.000000,35.000000,1.000000,2.000000,1.000000,0.000000,3.000000,4.000000,1.000000,0.000000,0.000000,457.000000,40.000000,28.000000,0.000000,4.000000,5.000000,4.000000,4.000000,1.000000,3.000000,37.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,6.000000,7.000000,5.000000,5.000000,-87.621900,39.663600
75%,205990.750000,2.000000,3.000000,120.000000,1800.000000,44.000000,2.000000,2.000000,2.000000,0.000000,5.000000,5.000000,1.000000,1.000000,0.000000,957.000000,50.000000,35.000000,31.000000,4.000000,6.000000,5.000000,5.000000,1.000000,5.000000,41.00000,1.000